In [ ]:
from decimal import *
getcontext().prec = 40

class Curve:
  """
  Python model of HMM token swap math.
  """

  def __init__(self, x0, y0, c, i):
    """
    x0: current token x balance in pool
    y0: current token y balance in pool
    c: compensation coefficient
    i: oracle price
    """
    self.x0 = Decimal(x0)
    self.y0 = Decimal(y0)
    self.c = c
    self.i = Decimal(i)

  def quantize(self, value, places=12):
    q = Decimal(10) ** -places
    return value.quantize(q)

  def to_int(decimal):
    return int(decimal.to_integral())

  def k(self):
    """
    k for a constant product curve
    """
    k = self.x0 * self.y0
    return k

  def sim_k(self):
    return int(self.k().to_integral())

  def delta_y_amm(self, delta_x):
    """
    Δy = K/X₀ - K/(X₀ + Δx)
    """
    k = self.k()
    return k/self.x0 - k/(self.x0 + delta_x)

  def sim_delta_y_amm(self, delta_x):
    return int(self.delta_y_amm(delta_x).to_integral())

  def delta_x_amm(self, delta_y):
    """
    Δy = K/Y₀ - K/(Y₀ + Δy)
    """
    k = self.k()
    return k/self.y0 - k/(self.y0 + delta_y)

  def sim_delta_x_amm(self, delta_y):
    return int(self.delta_x_amm(delta_y).to_integral())

  def swap_x_to_y_amm(self, delta_x):
    """
    swap x to y for a constant product curve given delta_x
    """
    k = self.k()
    x_new = self.x0 + delta_x
    y_new = k/x_new
    delta_x = x_new - self.x0
    delta_y = self.y0 - y_new
    return x_new, delta_x, y_new, delta_y

  def sim_swap_x_to_y_amm(self, delta_x):
    result = tuple(map(lambda d: int(d), self.swap_x_to_y_amm(delta_x)))
    return tuple(result)

  def xi(self):
    """
    Xᵢ = √(K/i)
    """
    k = self.k()
    return (k/self.i).sqrt()

  def sim_xi(self):
    return int(self.xi())

  def integral_c_eq_1(self, delta_x):
      """
      Δy = K/Xᵢ * LN(X₀/X_new)
      """
      k = self.k()
      xi = self.xi()
      x0 = self.x0
      x_new = self.x0 + delta_x
      log_x0 = self.x0.ln()
      # we flip this around to log(b) - log(a) so we can get the absolute difference
      # rather than a signed / negative number
      log_x_new = x_new.ln()
      log_delta_x = log_x_new - log_x0

      return k/xi * log_delta_x

  def sim_integral_c_eq_1(self, delta_x):
    return int(self.integral_c_eq_1(delta_x).to_integral(rounding=ROUND_CEILING))

  def integral_c_not_eq_1(self, delta_x):
      """
      Δy = K/[Xᵢᶜ/(c-1)] * [X₀ᶜ⁻¹ - X_newᶜ⁻¹]dX
      
      original using floats:
      k/(qi**c)/(c-1) * (q0**(c-1) - q_new**(c-1))

      proposed using unsigned decimal:
      c_sub_one_float = c - 1.0
      c_sub_one_decimal = Decimal(c_sub_one_float)
      c_sub_one_abs = c_sub_one_decimal.copy_abs()

      ((k/qi**c)/c_sub_one_abs) * ((1/q0**c_sub_one_abs) - (1/q_new**c_sub_one_abs))
      """
      k = self.k()
      qi = self.xi()
      c = Decimal(self.c)
      c_sub_one_float = self.c - 1.0
      c_sub_one_decimal = Decimal(c_sub_one_float)
      c_sub_one_abs = c_sub_one_decimal.copy_abs()
      q0 = self.x0
      q_new = self.x0 + delta_x
      return ((k/qi**c)/c_sub_one_abs) * ((1/q0**c_sub_one_abs) - (1/q_new**c_sub_one_abs))
      
  def sim_integral_c_not_eq_1(self, delta_x):
    return int(self.integral_c_not_eq_1(delta_x).to_integral(rounding=ROUND_CEILING))

In [ ]:
curve = Curve(1000, 1000, 0.5, 0.5)
print("AMM k: ", curve.k())
print("AMM delta_y_amm:", curve.delta_y_amm(10))
print("AMM delta_x_amm:", curve.delta_x_amm(50))
print("AMM swap_x_to_y_amm:", curve.swap_x_to_y_amm(50))
print("AMM swap_x_to_y_amm:", curve.sim_swap_x_to_y_amm(50))
print("xi:", curve.xi(), curve.sim_xi())

curve = Curve(151, 38, 1, 200)
delta_x = 50
print("integral_c_eq_1:", curve.integral_c_eq_1(delta_x),
      curve.sim_integral_c_eq_1(delta_x))

curve = Curve(1000, 1000, 2, 200)
print("integral_c_not_eq_1:", curve.integral_c_not_eq_1(delta_x), 
      curve.sim_integral_c_not_eq_1(delta_x))

curve = Curve(1000, 1000, 0.5, 200)
print("integral_c_not_eq_1:", curve.integral_c_not_eq_1(delta_x), 
      curve.sim_integral_c_not_eq_1(delta_x))

curve = Curve(1000, 1000, 0.1, 200)
print("integral_c_not_eq_1:", curve.integral_c_not_eq_1(delta_x), 
      curve.sim_integral_c_not_eq_1(delta_x))
